# Hermes — Run a basic Hartree-Fock energy calculation

See [the tutorial](/Tutorials/hermes_hartree_fock_energy_calculation.ipynb) for an in-depth explanation of how the Hermes Hartree-Fock energy calculation works.

In [11]:
# |hide
!rm -r ~/qdx/hermes_hartree_fock_quickstart/
!mkdir -p  ~/qdx/hermes_hartree_fock_quickstart/
%cd ~/qdx/hermes_hartree_fock_quickstart/

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/home/dylan/qdx/hermes_hartree_fock_quickstart


In [12]:
# Get a pdb to work with - we use the pdb-tools cli here but you can download directly from rcsb.org
!pdb_fetch 1brs | pdb_selchain -A | pdb_delhetatm > 1B39_A_nohet.pdb
!ls

1B39_A_nohet.pdb


In [14]:
import rush
import json
from pathlib import Path

# Get our client, for calling modules and using the rush API
# - edit in your access token and API URL or set the RUSH_TOKEN and RUSH_URL environment variables
client = rush.build_blocking_provider_with_functions(
    batch_tags=["hermes"],
)

(prepared_protein_qdxf, prepared_protein_pdb) = client.prepare_protein(
    Path(Path.home() / 'qdx' / 'hermes_hartree_fock_quickstart' /  "1B39_A_nohet.pdb")
)

# There may be multiple conformers, so select the first one
(first_conformer,) = client.pick_conformer(prepared_protein_qdxf, 0)

# Fragment the protein so that we can run a quantum energy calculation
(fragmented_protein,) = client.fragment_aa(first_conformer, 1, "All")

# Arguments
HERMES_RESOURCES = {
    "gpus": 1,
    "storage": 100,
    "storage_units": "MB",
    "walltime": 60,
}

fragmented_protein_out = await fragmented_protein.get()
fragmented_protein_contents = json.load(
    open(fragmented_protein.download_blocking(), "r")
)

# Quantum energy calculation
(hermes_energy,) = client.hermes_energy(
    fragmented_protein,
    {},
    {
        "basis": "STO-3G",
        "aux_basis": "6-31G",
        "method": "RestrictedRIMP2",
    },  # configuration for a fast converging, low accuracy run
    {
        "scf": {
            "max_iter": 50,
            "max_diis_history_length": 12,
            "convergence_metric": "DIIS",
            "dynamic_screening_threshold_exp": 10,
            "ndiis": 8,
            "niter": 40,
            "scf_conv": 0.000001,
        },
        "frag": {
            "cutoffs": {"dimer": 22},
            "cutoff_type": "Centroid",
            "level": "Dimer",
            "reference_fragment": len(
                fragmented_protein_contents["topology"]["fragments"]
            )
            - 1,
        },
    },
    resources=HERMES_RESOURCES,
)

energy = json.load(open(hermes_energy.download_blocking(), "r"))
# Let's look at the expanded Hartree Fock energy
energy["expanded_hf_energy"]

FileNotFoundError: [Errno 2] No such file or directory: '/home/dylan/qdx/hermes_hatree_fock_quickstart/1B39_A_nohet.pdb'